In [2]:
import json
class jsonl:
    def read(self, path):
        with open(path) as f:
            all_sentences = [json.loads(line) for line in f.readlines()]
        return all_sentences

    def write(self, path, data):
        with open(path, "w") as f:
            for line in data:
                f.write(json.dumps(line) + "\n")


afin_train = jsonl().read("./data/afin/train.jsonl")
afin_test = jsonl().read("./data/afin/test.jsonl")
afin_dev = jsonl().read("./data/afin/dev.jsonl")

afin_train_extra = jsonl().read("./data/afin/train-extra.jsonl")
afin_test_extra = jsonl().read("./data/afin/test-extra.jsonl")
afin_dev_extra = jsonl().read("./data/afin/dev-extra.jsonl")

large_afin = jsonl().read("./data/large-afin/large-afin.jsonl")
large_afin_extra = jsonl().read("./data/large-afin/large-afin-extra.jsonl")


def apply_on_all(func):
    all_orig = 0
    true_orig = 0

    all_extra = 0
    true_extra = 0

    for data in [afin_train, afin_test, afin_dev, large_afin]:
        temp1, temp2 = func(data)
        all_orig += temp1
        true_orig += temp2

    for data in [afin_train_extra, afin_test_extra, afin_dev_extra, large_afin_extra]:
        temp1, temp2 = func(data)
        all_extra += temp1
        true_extra += temp2

    print(f"all_orig: {all_orig}, true_orig: {true_orig}, all_extra: {all_extra}, true_extra: {true_extra}")
    print(f"orig: {true_orig/all_orig}, extra: {true_extra/all_extra}")

In [3]:
def cueCounter(dataset):
    if 'pi' in dataset[0]:
        affirs = [line['pi'] for line in dataset]
    else:
        affirs = [line['affirmative_interpretation'] for line in dataset]
    all_ = 0
    cued = 0
    for affir in affirs:
        all_ += 1
        if 'not' in affir:
            cued += 1
    return all_, cued

In [7]:
temp1, temp2 = cueCounter(afin_train_extra)
print(f"train: {temp2/temp1}")
temp1, temp2 = cueCounter(afin_test_extra)
print(f"test: {temp2/temp1}")
temp1, temp2 = cueCounter(afin_dev_extra)
print(f"dev: {temp2/temp1}")

train: 0.5790229885057471
test: 0.59392575928009
dev: 0.5503355704697986


In [4]:
apply_on_all(cueCounter)

all_orig: 156274, true_orig: 5013, all_extra: 292943, true_extra: 101599
orig: 0.03207827277730141, extra: 0.34682173665184013


In [5]:
print(len(large_afin_extra))
print(len(large_afin))

286984
153273


In [20]:
def createMap(orig, extra):
    all_ = 0
    check = 0
    missed = 0
    all_map = []
    for line in orig:
        all_ += 1
        temp = {}
        temp['orig'] = line
        for line2 in extra:
            if line['sentence'] == line2['sentence'] and line['affirmative_interpretation'].strip() != line2['affirmative_interpretation'].strip():
                temp['extra'] = line2
                check += 1
                break
        if not 'extra' in temp:
            temp['extra'] = None
            missed += 1
        all_map.append(temp)
        print(f"all: {all_}, check: {check}, missed: {missed}", end='\r')
    return all_map

In [ ]:
afin_test_all = createMap(afin_test, afin_test_extra)
print('finished test')
afin_train_all = createMap(afin_train, afin_train_extra)
print('finished train')
afin_dev_all = createMap(afin_dev, afin_dev_extra)
print('finished dev')

jsonl().write("./data/afin/test-all.jsonl", afin_test_all)
jsonl().write("./data/afin/train-all.jsonl", afin_train_all)
jsonl().write("./data/afin/dev-all.jsonl", afin_dev_all)

In [ ]:
large_afin_all = createMap(large_afin, large_afin_extra)
jsonl().write("./data/large-afin/large-afin-all.jsonl", large_afin_all)

In [2]:
import json
class jsonl:
    def read(self, path):
        with open(path) as f:
            all_sentences = [json.loads(line) for line in f.readlines()]
        return all_sentences

    def write(self, path, data):
        with open(path, "w") as f:
            for line in data:
                f.write(json.dumps(line) + "\n")

In [7]:
import pickle
with open("negations.pkl", "rb") as f:
    negation_cues = pickle.load(f)



def createNewExtra(all_data, neg_cues):
    new_extra = []
    # shuffle all_data
    import random
    random.shuffle(all_data)
    all_ = 0
    check = 0
    neg_found = 0
    for line in all_data:
        all_ += 1
        if line['extra'] != None:
            for cue in neg_cues:
                if " " + cue + " " in line['extra']['sentence']:
                    neg_found += 1
                    if " " + cue + " " in line['extra']['affirmative_interpretation']:
                        new_extra.append(line['extra'])
                        check += 1
                    break
                

    print(f"all: {all_}, check: {check}")
    print(f"all: {all_}, neg found: {neg_found}")
    random.shuffle(new_extra)
    return new_extra

'''
afin_test_all = jsonl().read("./data/afin/test-all.jsonl")
afin_train_all = jsonl().read("./data/afin/train-all.jsonl")
afin_dev_all = jsonl().read("./data/afin/dev-all.jsonl")

afin_test_new_extra = createNewExtra(afin_test_all, negation_cues)
afin_train_new_extra = createNewExtra(afin_train_all, negation_cues)
afin_dev_new_extra = createNewExtra(afin_dev_all, negation_cues)
jsonl().write("./data/afin/test-extra.jsonl", afin_test_new_extra)
jsonl().write("./data/afin/train-extra.jsonl", afin_train_new_extra)
jsonl().write("./data/afin/dev-extra.jsonl", afin_dev_new_extra)
'''

large_afin_all = jsonl().read("./data/large-afin/large-afin-all.jsonl")
large_afin_new_extra = createNewExtra(large_afin_all, negation_cues)
jsonl().write("./data/large-afin/large-afin-extra.jsonl", large_afin_new_extra)


all: 450, check: 251
all: 450, neg found: 399
all: 2101, check: 1144
all: 2101, neg found: 1855
all: 450, check: 233
all: 450, neg found: 401


'\nlarge_afin_all = jsonl().read("./data/large-afin/large-afin-all.jsonl")\nlarge_afin_new_extra = createNewExtra(large_afin_all)\njsonl().write("./data/large-afin/large-afin-extra.jsonl", large_afin_new_extra)\n'